In [ ]:

from litellm import completion
import time
gpt_model = "gpt-4.1-mini"
#claude_model = "claude-opus-4-20250514"
claude_model = "claude-3-5-haiku-latest"
#claude_model = "o4-mini"
import os

import chess
board = chess.Board()

import chess.svg
from IPython.display import display, HTML, clear_output,display_svg


import matplotlib.pyplot as plt
import base64
from io import BytesIO


metrics = {
    "move_number": [],
    "player": [],
    "context_size": [],
    "illegal_cumulative": [],
}


def update_graph():
    if len(metrics["move_number"]) < 2:
        return ""

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(5, 4))

    ax1.plot(metrics["move_number"], metrics["context_size"], color="red", linewidth=2)
    ax1.set_title("Context Size Per Move", fontsize=10)
    ax1.set_xlabel("Move")
    ax1.set_ylabel("Characters")
    ax1.grid(True, alpha=0.3)

    ax2.plot(metrics["move_number"], metrics["illegal_cumulative"], color="orange", linewidth=2)
    ax2.set_title("Cumulative Illegal Attempts", fontsize=10)
    ax2.set_xlabel("Move")
    ax2.set_ylabel("Illegal Attempts")
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    buf = BytesIO()
    fig.savefig(buf, format='png', dpi=100)
    plt.close(fig)
    buf.seek(0)
    img_base64 = base64.b64encode(buf.read()).decode('utf-8')
    return f'<img src="data:image/png;base64,{img_base64}" />'


gpt_system = """You are a world-class chess player named PLAYER_GPT playing White.
You will receive the current board state and a list of legal moves.
Think Carefully first and Respond with ONLY your next move in Standard Algebraic Notation (SAN).

Output ONLY the move. No explanation, no board, no commentary.
Example valid output: e4
Example valid output: Nf3"""

claude_system = """You are a world-class chess player named PLAYER_CLAUDE playing Black.
You will receive the current board state and a list of legal moves.
Think Carefully first and Respond with ONLY your next move in Standard Algebraic Notation (SAN).

Output ONLY the move. No explanation, no board, no commentary.
Example valid output: e5
Example valid output: Nf6"""

move_count=0
illegal_count=0

def call_gpt(gpt_system):
    global move_count, illegal_count
    legal_moves = [board.san(m) for m in board.legal_moves]
    user_msg = f"Current board:\n{str(board)}\n\nYour legal moves: {', '.join(legal_moves)}\n\nPick the best move."
    messages = [
        {"role": "system", "content": gpt_system},
        {"role": "user", "content": user_msg}
    ]

    move_count += 1
    input_tokens = sum(len(m["content"]) for m in messages)

    for attempt in range(10):
        response = completion(model=gpt_model, messages=messages)
        move = response.choices[0].message.content.strip().split('\n')[0].split()[0]
        try:
            board.push_san(move)

            # Track metrics
            metrics["move_number"].append(move_count)
            metrics["player"].append("GPT")  # or "Claude"
            metrics["context_size"].append(input_tokens)
            metrics["illegal_cumulative"].append(illegal_count)


            clear_output(wait=True)
            svg_str = chess.svg.board(board, lastmove=board.peek(), size=350)
            graph_html = update_graph()
            stats = f"""
            <div style="font-family: monospace; font-size: 14px; margin-bottom: 10px;">
                 <b>♟️ StateLess and Legal </b><br>
                 Move {move_count} | GPT plays: {move}<br>
                 Context size: ~{input_tokens} chars | History messages: {len(messages)}<br>
                 Illegal attempts so far: {illegal_count}<br>
            </div>
            """
            layout = f"""
            {stats}
            <div style="display: flex; align-items: margin-top: -20x; gap: 20px;">
                <div>{svg_str}</div>
                <div>{graph_html}</div>
            </div>
            """
            display(HTML(layout))
            return not board.is_game_over()
        except Exception as e:
            illegal_count += 1
            print(f"GPT invalid move '{move}' (attempt {attempt+1}): {e}")
            messages.append({"role": "assistant", "content": move})
            messages.append({"role": "user", "content": f"'{move}' is illegal. Pick from: {', '.join(legal_moves)}"})
    print("GPT failed 10 attempts, ending game.")
    return False

def call_claude(claude_system):
    global move_count, illegal_count
    legal_moves = [board.san(m) for m in board.legal_moves]
    user_msg = f"Current board:\n{str(board)}\n\nYour legal moves: {', '.join(legal_moves)}\n\nPick the best move."
    messages = [
        {"role": "system", "content": claude_system},
        {"role": "user", "content": user_msg}
    ]

    move_count += 1
    input_tokens = sum(len(m["content"]) for m in messages)

    for attempt in range(10):
        response = completion(model=claude_model, messages=messages)
        move = response.choices[0].message.content.strip().split('\n')[0].split()[0]
        try:
            board.push_san(move)

            # Track metrics
            metrics["move_number"].append(move_count)
            metrics["player"].append("GPT")  # or "Claude"
            metrics["context_size"].append(input_tokens)
            metrics["illegal_cumulative"].append(illegal_count)


            clear_output(wait=True)
            svg_str = chess.svg.board(board, lastmove=board.peek(), size=350)
            graph_html = update_graph()
            stats = f"""
            <div style="font-family: monospace; font-size: 14px; margin-bottom: 10px;">
                 <b>♟️ StateLess and Legal </b><br>
                 Move {move_count} | Claude plays: {move}<br>
                 Context size: ~{input_tokens} chars | History messages: {len(messages)}<br>
                 Illegal attempts so far: {illegal_count}<br>
            </div>
            """
            layout = f"""
            {stats}
            <div style="display: flex; align-items: margin-top: -20x; gap: 20px;">
                <div>{svg_str}</div>
                <div>{graph_html}</div>
            </div>
            """
            display(HTML(layout))
            return not board.is_game_over()
        except Exception as e:
            illegal_count += 1
            print(f"Claude invalid move '{move}' (attempt {attempt+1}): {e}")
            messages.append({"role": "assistant", "content": move})
            messages.append({"role": "user", "content": f"'{move}' is illegal. Pick from: {', '.join(legal_moves)}"})
    print("Claude failed 10 attempts, ending game.")
    return False

while True:
    if not call_gpt(gpt_system):
        break
    time.sleep(4)
    if not call_claude(claude_system):
        break
    time.sleep(4)
svg_str = chess.svg.board(board, lastmove=board.peek(), size=350)
if board.is_checkmate():
    winner = "Black (Claude)" if board.turn == chess.WHITE else "White (GPT)"
    result = f"♚ CHECKMATE! {winner} wins!"
elif board.is_stalemate():
    result = "½ STALEMATE! Draw."
elif board.is_insufficient_material():
    result = "½ DRAW — Insufficient material."
else:
    result = "Game ended."

stats = f"""
<div style="font-family: monospace; font-size: 14px; margin-bottom: 10px;">
    <b>♟️ STATELESS + LEGAL MOVES</b><br>
    <b style="font-size: 18px;">{result}</b><br>
    Total moves: {move_count} | Total illegal attempts: {illegal_count}<br>
</div>
"""
display(HTML(stats + svg_str))